# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [25]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [26]:
# 2) seu código aqui
sinasc.isnull().sum()

,0
ORIGEM,0
CODESTAB,115
CODMUNNASC,0
LOCNASC,0
IDADEMAE,0
...,...
munResUf,0
munResLat,1
munResLon,1
munResAlt,1


In [27]:
# 3) seu código aqui
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'CONSULTAS', 'APGAR5']
variaveis_interesse = sinasc[colunas_interesse]
variaveis_interesse.isnull().sum()

,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,317
ESCMAE,312
QTDFILVIVO,1573
GESTACAO,1232
CONSULTAS,0
APGAR5,103


In [29]:
# 4) seu código aqui
arquivo_limpo = variaveis_interesse[~variaveis_interesse.APGAR5.isna()]
print(arquivo_limpo.shape)
arquivo_limpo.isnull().sum()


(26925, 8)


,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,315
ESCMAE,310
QTDFILVIVO,1566
GESTACAO,1216
CONSULTAS,0
APGAR5,0


In [30]:
# 5) seu código aqui
arquivo_limpo['ESTCIVMAE'].fillna(9, inplace = True)
arquivo_limpo['CONSULTAS'].fillna(9, inplace = True)
arquivo_limpo

<ipython-input-30-b3834e429282>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  arquivo_limpo['ESTCIVMAE'].fillna(9, inplace = True)
<ipython-input-30-b3834e429282>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo_limpo['ESTCIVMAE'].fillna(9, inplace = True)
<ipython-input-30-b3834e429282>:3: FutureWarning: A value is trying to be set

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,10.0
...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,4,9.0


In [31]:
# 6) Seu código aqui
arquivo_limpo['QTDFILVIVO'].fillna(0, inplace=True)
arquivo_limpo

<ipython-input-31-0d8c5ad2e4f8>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  arquivo_limpo['QTDFILVIVO'].fillna(0, inplace=True)
<ipython-input-31-0d8c5ad2e4f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo_limpo['QTDFILVIVO'].fillna(0, inplace=True)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,10.0
...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,4,9.0


In [34]:
# 7) seu código aqui
#Jutificativa: Para decidir qual categoria por, eu fui atrás da média de escolaridade e tempo de gestação no portal do governo federal e IBGE.
#(https://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/28285-pnad-educacao-2019-mais-da-metade-das-pessoas-de-25-anos-ou-mais-nao-completaram-o-ensino-medio)

arquivo_limpo['ESCMAE'].fillna(4, inplace=True)
arquivo_limpo['GESTACAO'].fillna(5, inplace=True)
arquivo_limpo.isnull().sum()

<ipython-input-34-e02743b30630>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo_limpo['ESCMAE'].fillna(4, inplace=True)
<ipython-input-34-e02743b30630>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arquivo_limpo['GESTACAO'].fillna(5, inplace=True)


,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,0
ESCMAE,0
QTDFILVIVO,0
GESTACAO,0
CONSULTAS,0
APGAR5,0


In [38]:
# 8) seu código
arquivo_limpo.loc[arquivo_limpo["APGAR5"] <= 3, "CAT_APGAR5"] = "ASFIXIA_SEVERA"
arquivo_limpo.loc[(arquivo_limpo["APGAR5"] >= 4) & (arquivo_limpo["APGAR5"] <=5), "CAT_APGAR5"] = "ASFIXIA_MODERADA"
arquivo_limpo.loc[(arquivo_limpo["APGAR5"] >= 6) & (arquivo_limpo["APGAR5"] <=7), "CAT_APGAR5"] = "ASFIXIA_LEVE"
arquivo_limpo.loc[arquivo_limpo["APGAR5"] >= 8 ,"CAT_APGAR5"] = "NORMAL"

arquivo_limpo.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,CONSULTAS,APGAR5,CAT_APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,4,10.0,NORMAL
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,9.0,NORMAL
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,4,10.0,NORMAL
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,3,10.0,NORMAL
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,10.0,NORMAL


In [41]:
# 9) seu código aqui
arquivo_limpo.columns = ['loc_nasc','idade_mae','est_civ_mae','esc_mae','qtd_fil_vivo','gestacao','consultas','apgar_5','cat_apgar_5']


In [42]:
arquivo_limpo.columns

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'consultas', 'apgar_5', 'cat_apgar_5'],
      dtype='object')

In [43]:
arquivo_limpo.head()

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,consultas,apgar_5,cat_apgar_5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,4,10.0,NORMAL
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,9.0,NORMAL
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,4,10.0,NORMAL
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,3,10.0,NORMAL
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,4,10.0,NORMAL
